# Getting Started

In [ ]:
## Install the required packages in case you have not already installed it
# (In case you cannot find the environment you've created in another folder, try copying the whole .venv folder to this current directory)
!pip install numpy scipy pysr pysindy pandas matplotlib notebook

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from src.utils import *

## Uncomment the following lines if you want to use PySINDy or PySR
# import pysindy as ps
# from pysr import PySRRegressor

print("All packages imported successfully!")

All packages imported successfully!


## Importing trajectories

The trajectory measured usually comes in csv file.
- First column represent time
- Next three column represents the cartesian coordinate (relative to earth) of the satalite's position (in m)
- Last three column represents the velocity (in m/s)
- Each row is data taken in each time point

Let's start by importing the file

In [2]:
# Load the CSV file with correct column extraction
# Assuming the first column is time, next three are position (r1, r2, r3), last three are velocity (v1, v2, v3)
df = pd.read_csv('./data/Challenge1.csv', header=0, usecols=[0,1,2,3,4,5,6], names=['time','r1', 'r2', 'r3', 'v1', 'v2', 'v3'])

# Extract position and velocity arrays
# Convert time strings to pandas datetime
t_raw = pd.to_datetime(df['time'])
# Calculate seconds relative to the first time point
t = (t_raw - t_raw.iloc[0]).dt.total_seconds().values # Time in seconds relative to first time point
r = df[['r1', 'r2', 'r3']].values
v = df[['v1', 'v2', 'v3']].values

## Convert between Cartesian Coordinate, Keplarian Elements and Equinoctial Elements

Any conversion between Cartesian $\mathbf{r}$ and $\mathbf{v}$ requires also the knowledge of the gravitational constant `mu`.

In [ ]:
mu = 3.986004414498200e14  # Central Body's gravitational constant (m^3/s^2) - from parameter.csv
Re = 6.378136460000000e6  # Central Body's equatorial radius (m) - from parameter.csv

To convert it to Keplerian Elements

In [ ]:
kepl = ijk2keplerian(r,v,mu)

# To extract the individual elements
a = kepl[..., 0]        # a: semi-major axis 
e = kepl[..., 1]        # e: eccentricity 
incl = kepl[..., 2]     # incl: inclination 
raan = kepl[..., 3]     # raan: right ascension of ascending node 
argp = kepl[..., 4]     # argp: argument of perigee 
nu = kepl[..., 5]       # nu: true anomaly 


To convert to Equinoctial Elements

In [ ]:
equi = ijk2equinoctial(r,v,mu)

# To extract the individual elements
p = equi[..., 0] # p: semi-parameter of the orbit
f = equi[..., 1] # f: e cos(raan + argp)
g = equi[..., 2] # g: e sin(raan + argp)
h = equi[..., 3] # h: tan(incl/2)cos(raan)
k = equi[..., 4] # k: tan(incl/2)sin(raan)
L = equi[..., 5] # L: true longitude

To get the initial condition (notice the shape - the `odeint` in `SciPy` takes initial condition in this format)

In [ ]:
equi0 = equi[0]
kepl0 = kepl[0]

equi0.shape

(6,)

We can also convert between Keplerian and Equinoctial

In [ ]:
kepl2 = equinoctial2keplerian(equi)
equi2 = keplerian2equinoctial(kepl)

Finally, if we want to convert back to Cartesian

In [ ]:
r2, v2 = keplerian2ijk(kepl, mu)

# Learning models from the data

## Approach 1: Directly fit the time-series solution $x(t)$

Things to think about:
- What coordinate system/ representation would you use?
- What are the variables each elements should be represented in? 
- What operators would you choose?

In [ ]:
# (insert your code here)
# (hint: Which coordinate would you use fit the data?)
    # The original Cartesian coordinates (x, y, z)?
    # The Keplerian elements (a, e, i, Ω, ω, ν)?
    # The Equinoctial elements (p, f, g, h, k, L)?

# (hint: What dictionary term would you use to fit the data?)

## Approach 2: Fitting the operator $dx/dt=f(x)$ that creates the time-series
Instead of fitting $x(t)$, we can also fit $f(x)$ that generate the data in the first place. Before proceeding, think about:
- Are we sure the system is invariant in time? i.e. $f(x)$ does not depend on time?
- What are the pros and cons of fitting $f(x)$ instead of $x(t)$?
  - What if I want the model to be applied to a different initial condition?
  - Do you expect the solution coming out of fitting $f(x)$ follow the data in the long term? Why?
- What do I want to get out of this dataset? A generalisable model? Physical Understaning? Or just forecasting?

###  Approximating time derivatives
To fit $f(x)$, we first need an approximation of $dx/dt$.

In [28]:
from scipy.sparse import spdiags

## Setting up a 2nd order finite difference matrix
N_pt = kepl.shape[0]  # Number of points in the time series
dt = t[1] - t[0]
I  = spdiags(np.array([[0], [1],[0]]) @ np.ones((1,N_pt)) , np.array([0,1,2]), m = N_pt - 2, n = N_pt)
D1 = spdiags(np.array([[-1/2], [0],[1/2]]) @ np.ones((1,N_pt)) / dt, np.array([0,1,2]), m = N_pt - 2, n = N_pt)

## Applying to keplerian elements
d_kepl_dt = D1 @ kepl # Time derivative of keplerian elements 
kepl_ = I @ kepl # The keplerian elements value at the time point of the derivative

## Applying to equinoctial elements
d_equi_dt = D1 @ equi # Time derivative of equinoctial elements 
equi_ = I @ equi # The equinoctial elements value at the time point of the derivative

You may want to use a more accurate way to find the time derivative!
(Also check out the time derivative utility in PySINDy!)

###  Finding $F_R$, $F_S$ and $F_W$ from the time derivative
Given our prior knowledge of the system, we can further reduce the problem to just finding $F_R$, $F_S$ and $F_W$ in terms of the state $x$ based on known transformation between the time derivatives and the perturbation forces on the satellite.
- What are the advantages of fitting $F_R$, $F_S$ and $F_W$ instead of the time derivative?

How would you go about recovering $F_R$, $F_S$ and $F_W$ from the approximation of the time derivative above?

In [ ]:
# (insert your code here)
# (hint: The eqn. to transform  FR FS and FW into d (element)/ dt can be written as:
#     d (element)/ dt = A(element,mu) * F_RSW(element) + b(element,mu)
#     The function A and b can be found in propagation.py
# (hint: look at EquinoctialPropagation.ipynb or KeplerianPropagation.ipynb to see how the derivatives are computed from FR FS and FW)!